# Olympic Athletes

Potential Questions:


We'll start by loading the datafile into a Pandas dataframe, and having a brief look at some of the data.

# Stuff to include:
* .rename(dict{'orig_name':'new_name'})
* .describe(), df['column'].describe()
* .value_counts()

* Get unique combinations of column values
* Create a new column from existing columns
* Join two datasets together
* .apply() to create a new column
* Plot some data

* Analysis suggestions to get them started

In [4]:
import pandas as pd
data = pd.read_csv("athlete_events.csv")
data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


As you can see, we have 15 columns of different values. If there were more, we might not be able to view them all in this brief synopsis, but we could have a look at them using the `data.columns` attribute.

Currently we don't know how much data there is, as `.head()` by default only shows the first 5 rows (you can show more by providing an argument, like `.head(10)`). Also, it looks like some of the columns have `NaN` results, which is the representation for a NumPy Not-a-Number token (i.e. there is no data entry in that column for the row). We should have a broader look at the data to see if this will be a problem.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
ID        271116 non-null int64
Name      271116 non-null object
Sex       271116 non-null object
Age       261642 non-null float64
Height    210945 non-null float64
Weight    208241 non-null float64
Team      271116 non-null object
NOC       271116 non-null object
Games     271116 non-null object
Year      271116 non-null int64
Season    271116 non-null object
City      271116 non-null object
Sport     271116 non-null object
Event     271116 non-null object
Medal     39783 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [12]:
valid_rows = data[ ['ID', 'Age', 'Height', 'Weight'] ].count()
valid_rows - data.shape[0]

ID            0
Age       -9474
Height   -60171
Weight   -62875
dtype: int64

We can see that there are roughly 60,000 rows missing height and/or weight data, and only around 10,000 rows missing weight rows. This should theoretically leave us with ~200,000 rows that contain all data (we're counting an empty value in the "Medal" column as a valid value), but let's check just to make sure, by filtering out all rows that do not have a valid Age, Height, and Weight.

We can do this by using the `data.loc[]` function to select rows that meet a certain criteria. This criteria can involve many boolean truth operators (such as `==`, `>`, etc.) as well as some functions built into Pandas/Numpy that return either `True` or `False` for each row. For this example, we'll use the `.notna()` function, which returns `True` if the value is not NaN, and `False` if it is.

In [16]:
full_rows = data.loc[ (data['Age'].notna()) & (data['Weight'].notna()) & (data['Height'].notna()) ]
full_rows.shape

(206165, 15)

We can see that we do indeed have 206,165 valid rows that all have valid Age, Height, and Weight values.
However, we can also see from the `.head()` function above that there appears to be an entry for each athlete, in each event, in each year. These may overlap, so how many athletes do we actually have in the dataset? We can check this by using the `.unique()` function, which will return a list of all of the unique values in a given column, or Series' as they are called in Pandas.

In [19]:
num_athletes = len(data['Name'].unique())
print("There are {} unique athletes.".format(num_athletes))

There are 134732 unique athletes.


It is not unfeasible, however, that there existed different athletes with the same name. We can see that each athlete has an 'ID' number, which probably would have been a better marker for individual athletes.

In [25]:
real_num_athletes = len(data['ID'].unique())
print("There are ACTUALLY {} unique athletes.".format(real_num_athletes))

There are ACTUALLY 135571 unique athletes.


Okay, so there are more unique athletes than we initially thought, so some of them must share names. To have a look into this, we are going to use the `dataframe.groupby(X)` function. Here, X can be either the name of a column, or a list of names of multiple columns. This function will then return a series of (value, group) sets, where 'value' is the unique entry in the specified column (or a tuple of a unique combination of entries if more than one column was specified), while 'group' is the subgroup of the dataframe in which all rows have the returned values. For example:

In [32]:
for (team_value, year_value), group in data.groupby(["Team", "Year"]):
    print(team_value, year_value, len(group['ID'].unique()))

30. Februar 1952 2
A North American Team 1900 4
Acipactli 1964 3
Acturus 1948 2
Afghanistan 1936 15
Afghanistan 1948 25
Afghanistan 1956 12
Afghanistan 1960 12
Afghanistan 1964 8
Afghanistan 1968 5
Afghanistan 1972 8
Afghanistan 1980 11
Afghanistan 1988 5
Afghanistan 1996 2
Afghanistan 2004 5
Afghanistan 2008 4
Afghanistan 2012 6
Afghanistan 2016 3
Akatonbo 1964 3
Alain IV 1964 3
Albania 1972 5
Albania 1992 7
Albania 1996 7
Albania 2000 4
Albania 2004 7
Albania 2006 1
Albania 2008 11
Albania 2010 1
Albania 2012 9
Albania 2014 1
Albania 2016 6
Alcaid 1952 3
Alcyon-6 1900 1
Alcyon-7 1900 1
Aldebaran 1964 2
Aldebaran II 1960 2
Aletta 1960 3
Algeria 1964 1
Algeria 1968 3
Algeria 1972 5
Algeria 1980 54
Algeria 1984 33
Algeria 1988 42
Algeria 1992 39
Algeria 1996 45
Algeria 2000 47
Algeria 2004 61
Algeria 2006 2
Algeria 2008 56
Algeria 2010 1
Algeria 2012 38
Algeria 2016 64
Ali-Baba II 1948 7
Ali-Baba IV 1952 2
Ali-Baba IX 1964 2
Ali-Baba VI 1960 2
Allegro 1948 3
Almaz 1964 2
Aloha II 1924 3

Brazil 1952 92
Brazil 1956 45
Brazil 1960 68
Brazil 1964 57
Brazil 1968 76
Brazil 1972 81
Brazil 1976 79
Brazil 1980 106
Brazil 1984 147
Brazil 1988 160
Brazil 1992 189
Brazil 1994 1
Brazil 1996 213
Brazil 1998 1
Brazil 2000 190
Brazil 2002 10
Brazil 2004 235
Brazil 2006 9
Brazil 2008 260
Brazil 2010 5
Brazil 2012 237
Brazil 2014 13
Brazil 2016 452
Brazil-1 1996 4
Brazil-1 2000 4
Brazil-1 2004 4
Brazil-1 2008 4
Brazil-1 2012 6
Brazil-1 2016 6
Brazil-2 1996 4
Brazil-2 2000 4
Brazil-2 2004 4
Brazil-2 2008 4
Brazil-2 2012 6
Brazil-2 2016 6
Bremen 1936 1
Brentina 2008 1
Breslau 1936 1
Briar 2008 1
Brigantia 1964 2
British Virgin Islands 1984 10
British Virgin Islands 1988 3
British Virgin Islands 1992 4
British Virgin Islands 1996 7
British Virgin Islands 2000 1
British Virgin Islands 2004 1
British Virgin Islands 2008 2
British Virgin Islands 2012 2
British Virgin Islands 2014 1
British Virgin Islands 2016 4
Bronia 1960 3
Brunei 1996 1
Brunei 2000 2
Brunei 2004 1
Brunei 2012 3
Brunei 2016

Congo (Kinshasa) 1996 14
Congo (Kinshasa) 2000 2
Congo (Kinshasa) 2004 4
Congo (Kinshasa) 2008 5
Congo (Kinshasa) 2012 4
Congo (Kinshasa) 2016 4
Cook Islands 1988 7
Cook Islands 1992 2
Cook Islands 1996 3
Cook Islands 2000 2
Cook Islands 2004 3
Cook Islands 2008 4
Cook Islands 2012 8
Cook Islands 2016 9
Cornwall 1908 15
Costa Rica 1936 1
Costa Rica 1964 2
Costa Rica 1968 18
Costa Rica 1972 3
Costa Rica 1976 5
Costa Rica 1980 30
Costa Rica 1984 31
Costa Rica 1988 18
Costa Rica 1992 20
Costa Rica 1996 11
Costa Rica 2000 7
Costa Rica 2002 1
Costa Rica 2004 20
Costa Rica 2006 1
Costa Rica 2008 8
Costa Rica 2012 11
Costa Rica 2016 11
Cote d'Ivoire 1964 9
Cote d'Ivoire 1968 10
Cote d'Ivoire 1972 11
Cote d'Ivoire 1976 8
Cote d'Ivoire 1984 15
Cote d'Ivoire 1988 28
Cote d'Ivoire 1992 13
Cote d'Ivoire 1996 11
Cote d'Ivoire 2000 14
Cote d'Ivoire 2004 5
Cote d'Ivoire 2008 21
Cote d'Ivoire 2012 9
Cote d'Ivoire 2016 12
Covunco III 1956 2
Crabe I-11 1900 1
Crabe I-2 1900 1
Crabe I-3 1900 1
Crabe II-1

France-1 2016 4
France-2 1900 3
France-2 1908 11
France-2 1920 4
France-2 1924 10
France-2 1928 5
France-2 1936 6
France-2 1948 5
France-2 1952 2
France-2 1956 2
France-2 1968 6
France-2 1972 2
France-2 1976 2
France-2 1988 2
France-2 1992 12
France-2 1994 6
France-2 1996 4
France-2 1998 6
France-2 2000 2
France-2 2002 6
France-2 2004 4
France-2 2006 2
France-2 2008 2
France-2 2010 2
France-2 2012 2
France-2 2014 6
France-2 2016 3
France-3 1900 2
France-3 1920 3
France-3 1992 4
France-3 1998 2
France-4 1900 2
France/Great Britain 1900 2
Frankfurt Club 1900 15
Freddy 1936 2
Freia-19 1900 1
Freia-5 1900 1
Freja 1908 3
Frieda 1928 5
Frimousse 1900 1
Frip IV 1960 2
Fruits 1928 5
Gabbiano 1960 3
Gabon 1972 1
Gabon 1984 4
Gabon 1988 2
Gabon 1992 5
Gabon 1996 7
Gabon 2000 5
Gabon 2004 5
Gabon 2008 4
Gabon 2012 21
Gabon 2016 6
Galatea II 1952 3
Galejan 1960 3
Galerna 1948 3
Gallant 1932 5
Gallia II 1912 7
Galopin De La Font 2008 1
Galopin-20 1900 2
Galopin-9 1900 2
Galt Football Club 1904 13
G

Honduras 2004 5
Honduras 2008 25
Honduras 2012 25
Honduras 2016 23
Hong Kong 1952 4
Hong Kong 1956 2
Hong Kong 1960 4
Hong Kong 1964 36
Hong Kong 1968 11
Hong Kong 1972 10
Hong Kong 1976 25
Hong Kong 1984 47
Hong Kong 1988 47
Hong Kong 1992 38
Hong Kong 1996 23
Hong Kong 2000 31
Hong Kong 2002 2
Hong Kong 2004 30
Hong Kong 2006 1
Hong Kong 2008 34
Hong Kong 2010 1
Hong Kong 2012 41
Hong Kong 2014 1
Hong Kong 2016 37
Hong Kong-1 1988 2
Hong Kong-1 2004 4
Hong Kong-2 1988 2
Hong Kong-2 2004 4
Hornet 1952 2
Humbug V 1964 2
Hungaria 1928 5
Hungaria Evezos Egylet 1912 9
Hungary 1896 7
Hungary 1900 18
Hungary 1904 4
Hungary 1906 35
Hungary 1908 56
Hungary 1912 112
Hungary 1924 93
Hungary 1928 117
Hungary 1932 58
Hungary 1936 233
Hungary 1948 147
Hungary 1952 197
Hungary 1956 113
Hungary 1960 183
Hungary 1964 210
Hungary 1968 177
Hungary 1972 233
Hungary 1976 181
Hungary 1980 265
Hungary 1984 9
Hungary 1988 193
Hungary 1992 237
Hungary 1994 16
Hungary 1996 213
Hungary 1998 17
Hungary 2000 178

Liberia 2012 3
Liberia 2016 2
Libya 1968 1
Libya 1980 29
Libya 1988 6
Libya 1992 5
Libya 1996 5
Libya 2000 3
Libya 2004 8
Libya 2008 6
Libya 2012 4
Libya 2016 7
Liechtenstein 1936 10
Liechtenstein 1948 12
Liechtenstein 1952 2
Liechtenstein 1956 8
Liechtenstein 1960 8
Liechtenstein 1964 8
Liechtenstein 1968 11
Liechtenstein 1972 10
Liechtenstein 1976 15
Liechtenstein 1980 7
Liechtenstein 1984 17
Liechtenstein 1988 25
Liechtenstein 1992 14
Liechtenstein 1994 10
Liechtenstein 1996 2
Liechtenstein 1998 8
Liechtenstein 2000 2
Liechtenstein 2002 8
Liechtenstein 2004 1
Liechtenstein 2006 5
Liechtenstein 2008 2
Liechtenstein 2010 4
Liechtenstein 2012 3
Liechtenstein 2014 3
Liechtenstein 2016 3
Life boat naval ship "Spetsai"-1 1906 7
Linglom 1964 3
Lithuania 1924 13
Lithuania 1928 13
Lithuania 1992 53
Lithuania 1994 6
Lithuania 1996 61
Lithuania 1998 7
Lithuania 2000 61
Lithuania 2002 8
Lithuania 2004 59
Lithuania 2006 7
Lithuania 2008 71
Lithuania 2010 6
Lithuania 2012 62
Lithuania 2014 9
Lith

Nigeria-2 2004 4
Nikh 1956 2
Nina 1912 7
Nina Claire-2 1900 4
Nina Claire-30 1900 4
Nirefs 1960 3
Nirwana 1952 5
Nokaut II 1960 2
Nora 1948 2
Noreg 1928 4
Norman-1 1912 5
Norna 1928 4
Nortazo 1948 2
North Borneo 1956 2
North Korea 1964 13
North Korea 1972 43
North Korea 1976 38
North Korea 1980 47
North Korea 1984 6
North Korea 1988 6
North Korea 1992 82
North Korea 1996 23
North Korea 1998 8
North Korea 2000 31
North Korea 2004 36
North Korea 2006 6
North Korea 2008 58
North Korea 2010 2
North Korea 2012 52
North Korea 2016 31
North Korea-1 1992 4
North Korea-1 1996 2
North Korea-2 1992 4
North Korea-2 1996 2
North Yemen 1984 2
North Yemen 1988 8
Norway 1900 7
Norway 1904 3
Norway 1906 32
Norway 1908 55
Norway 1912 147
Norway 1920 134
Norway 1924 68
Norway 1928 68
Norway 1932 26
Norway 1936 86
Norway 1948 112
Norway 1952 153
Norway 1956 60
Norway 1960 61
Norway 1964 75
Norway 1968 111
Norway 1972 179
Norway 1976 103
Norway 1980 63
Norway 1984 161
Norway 1988 132
Norway 1992 162
Norway

Romania-2 1936 2
Romania-2 1956 4
Romania-2 1964 2
Romania-2 1968 2
Romania-2 1972 2
Romania-2 1976 6
Romania-2 1980 4
Romania-2 1984 2
Romania-2 1988 2
Romania-2 1992 4
Romania-2 1998 2
Romania-2 2002 2
Romania-2 2006 4
Romania-2 2010 2
Romania-2 2016 2
Romolo 1956 2
Rose Pompon 1920 3
Rostock 1936 1
Rowing Club Castillon-3 1900 3
Roy 1964 3
Royal Club Nautique de Gand 1900 9
Royal Club Nautique de Gand 1906 3
Royal Club Nautique de Gand 1908 9
Royal Club Nautique de Gand 1912 5
Rozenn-3 1900 1
Rush IV 1956 4
Rush V 1956 3
Rush VII 1964 3
Russia 1900 3
Russia 1908 6
Russia 1912 142
Russia 1924 3
Russia 1994 96
Russia 1996 390
Russia 1998 105
Russia 2000 435
Russia 2002 129
Russia 2004 443
Russia 2006 149
Russia 2008 453
Russia 2010 142
Russia 2012 424
Russia 2014 191
Russia 2016 279
Russia-1 1900 2
Russia-1 1994 8
Russia-1 1998 8
Russia-1 2002 11
Russia-1 2004 6
Russia-1 2006 10
Russia-1 2008 4
Russia-1 2010 14
Russia-1 2012 4
Russia-1 2014 12
Russia-1 2016 4
Russia-2 1994 8
Russia-2 

Spain-1 2008 4
Spain-1 2012 4
Spain-1 2016 4
Spain-2 1920 2
Spain-2 1924 4
Spain-2 1968 6
Spain-2 1996 2
Spain-2 2004 4
Spain-2 2008 4
Spain-2 2012 4
Spain-2 2016 4
Spirit III 1956 4
Spirit VI 1960 3
Springbok 1932 2
Squid III 1964 2
Sri Lanka 1948 7
Sri Lanka 1952 5
Sri Lanka 1956 3
Sri Lanka 1960 5
Sri Lanka 1964 6
Sri Lanka 1968 3
Sri Lanka 1972 4
Sri Lanka 1980 4
Sri Lanka 1984 4
Sri Lanka 1988 6
Sri Lanka 1992 11
Sri Lanka 1996 9
Sri Lanka 2000 18
Sri Lanka 2004 7
Sri Lanka 2008 8
Sri Lanka 2012 7
Sri Lanka 2016 9
St. Louis Amateur Athletic Association 1904 12
St. Louis Southwest Turnverein #1-2 1904 5
St. Louis Southwest Turnverein #2-3 1904 5
St. Margrite 1948 2
St. Rose-2 1904 12
Stade Franais AC-2 1900 2
Starita 1948 2
Starlight III 1956 2
State VI 1964 3
Stella-2 1920 3
Struten 1960 3
Studenternes Roklub-2 1912 5
Subbnboana 1964 4
Sudan 1960 10
Sudan 1968 5
Sudan 1972 26
Sudan 1984 7
Sudan 1988 8
Sudan 1992 6
Sudan 1996 4
Sudan 2000 3
Sudan 2004 4
Sudan 2008 9
Sudan 2012 6
Su

United States-1 2006 12
United States-1 2008 8
United States-1 2010 12
United States-1 2012 9
United States-1 2014 12
United States-1 2016 8
United States-10 1904 2
United States-11 1904 2
United States-12 1904 2
United States-13 1904 2
United States-14 1904 2
United States-2 1904 2
United States-2 1924 4
United States-2 1928 7
United States-2 1932 8
United States-2 1936 7
United States-2 1948 8
United States-2 1952 8
United States-2 1956 8
United States-2 1960 2
United States-2 1964 10
United States-2 1968 8
United States-2 1972 11
United States-2 1976 12
United States-2 1980 10
United States-2 1984 12
United States-2 1988 10
United States-2 1992 14
United States-2 1994 8
United States-2 1996 4
United States-2 1998 10
United States-2 2000 8
United States-2 2002 13
United States-2 2004 10
United States-2 2006 12
United States-2 2008 8
United States-2 2010 12
United States-2 2012 9
United States-2 2014 14
United States-2 2016 8
United States-3 1904 2
United States-3 1960 2
United States

Using `.groupby(X)` on multiple columns can be very useful if you are wanting to use a unique combination of several columns as an identified (e.g. if FirstName and Surname were in different columns). However, in many cases, such as the one above, instead of grouping by two columns you would probably want to consecutively group by a single column twice, as shown:

In [33]:
for team, team_group in data.groupby("Team"):
    print(team, ":")
    for year, year_team_group in team_group.groupby("Year"):
        print(year, ":", len(year_team_group['ID'].unique()))
    print()

30. Februar :
1952 : 2

A North American Team :
1900 : 4

Acipactli :
1964 : 3

Acturus :
1948 : 2

Afghanistan :
1936 : 15
1948 : 25
1956 : 12
1960 : 12
1964 : 8
1968 : 5
1972 : 8
1980 : 11
1988 : 5
1996 : 2
2004 : 5
2008 : 4
2012 : 6
2016 : 3

Akatonbo :
1964 : 3

Alain IV :
1964 : 3

Albania :
1972 : 5
1992 : 7
1996 : 7
2000 : 4
2004 : 7
2006 : 1
2008 : 11
2010 : 1
2012 : 9
2014 : 1
2016 : 6

Alcaid :
1952 : 3

Alcyon-6 :
1900 : 1

Alcyon-7 :
1900 : 1

Aldebaran :
1964 : 2

Aldebaran II :
1960 : 2

Aletta :
1960 : 3

Algeria :
1964 : 1
1968 : 3
1972 : 5
1980 : 54
1984 : 33
1988 : 42
1992 : 39
1996 : 45
2000 : 47
2004 : 61
2006 : 2
2008 : 56
2010 : 1
2012 : 38
2016 : 64

Ali-Baba II :
1948 : 7

Ali-Baba IV :
1952 : 2

Ali-Baba IX :
1964 : 2

Ali-Baba VI :
1960 : 2

Allegro :
1948 : 3

Almaz :
1964 : 2

Aloha II :
1924 : 3
1948 : 2

Amateur Athletic Association :
1900 : 5

American Samoa :
1988 : 6
1992 : 3
1994 : 2
1996 : 7
2000 : 4
2004 : 3
2008 : 4
2012 : 4
2016 : 4

Amolgavar :
19

2000 : 2
2004 : 1
2012 : 3
2016 : 3

Brussels Swimming and Water Polo Club :
1900 : 10

Brynhild-2 :
1900 : 1

Bu III :
1952 : 2

Bucintoro Venezia :
1906 : 5

Bucintoro Venezia-1 :
1906 : 3

Bulgaria :
1924 : 24
1928 : 5
1932 : 2
1936 : 33
1948 : 4
1952 : 73
1956 : 54
1960 : 105
1964 : 70
1968 : 118
1972 : 134
1976 : 187
1980 : 279
1984 : 16
1988 : 198
1992 : 165
1994 : 17
1996 : 110
1998 : 19
2000 : 91
2002 : 23
2004 : 95
2006 : 21
2008 : 70
2010 : 19
2012 : 63
2014 : 18
2016 : 51

Bulgaria-1 :
1988 : 2
1992 : 4

Bulgaria-2 :
1988 : 2
1992 : 4

Buraddoo :
1956 : 3
1960 : 3

Burevestnik :
1952 : 3

Burkina Faso :
1972 : 1
1988 : 6
1992 : 4
1996 : 5
2000 : 4
2004 : 5
2008 : 6
2012 : 5
2016 : 5

Burundi :
1996 : 7
2000 : 6
2004 : 7
2008 : 3
2012 : 6
2016 : 9

Buscape II :
1948 : 2

Bylina-2 :
1912 : 5

C.V.A.-14 :
1900 : 2

C.V.A.-7 :
1900 : 2

Calcinhas :
1960 : 2

Calimucho :
2008 : 1

Calypse II :
1960 : 2

Cambodia :
1956 : 2
1964 : 11
1972 : 9
1996 : 5
2000 : 4
2004 : 4
2008 : 4
20

Dylan-2 :
1908 : 2

EA II :
1936 : 6

East Germany :
1968 : 278
1972 : 327
1976 : 309
1980 : 384
1984 : 38
1988 : 297

East Germany-1 :
1968 : 4
1972 : 6
1976 : 10
1980 : 8
1984 : 10
1988 : 6

East Germany-2 :
1968 : 4
1972 : 6
1976 : 10
1980 : 10
1984 : 10
1988 : 8

East Germany-3 :
1972 : 2
1976 : 2
1984 : 2

Ecuador :
1924 : 3
1968 : 15
1972 : 2
1976 : 5
1980 : 12
1984 : 11
1988 : 13
1992 : 13
1996 : 19
2000 : 10
2004 : 16
2008 : 25
2012 : 36
2016 : 37

Edelweiss II-1 :
1920 : 3

Egypt :
1906 : 2
1920 : 18
1924 : 24
1928 : 32
1936 : 53
1948 : 85
1952 : 106
1956 : 3
1964 : 73
1968 : 30
1972 : 23
1976 : 26
1984 : 115
1988 : 49
1992 : 75
1996 : 29
2000 : 89
2004 : 97
2008 : 100
2012 : 109
2016 : 121

Eissero VI :
1952 : 2

El Salvador :
1932 : 1
1968 : 60
1972 : 11
1984 : 10
1988 : 6
1992 : 4
1996 : 8
2000 : 8
2004 : 7
2008 : 11
2012 : 10
2016 : 8

Eleda :
1920 : 7

Elisabeth V :
1924 : 3

Elisabeth X :
1952 : 5

Elsie :
1920 : 4

Elvis Va :
2008 : 1

Emily :
1924 : 5

Encore :
1952 : 

1900 : 1

Gyrinus-1 :
1908 : 3

Haiti :
1900 : 2
1924 : 8
1928 : 2
1932 : 3
1960 : 1
1972 : 7
1976 : 13
1984 : 3
1988 : 4
1992 : 7
1996 : 7
2000 : 5
2004 : 8
2008 : 7
2012 : 5
2016 : 10

Hakahana :
1960 : 2

Hamburg :
1936 : 1

Hannover :
1936 : 1

Harmony :
1960 : 2

Hatuey :
1924 : 3

Hayama :
1964 : 2

Hb-20 :
1900 : 1

Hb-24 :
1900 : 1

Heatherbell :
1912 : 7

Heidelberg :
1936 : 1

Heira II :
1920 : 9

Helsingfors R. K. :
1912 : 5

Hera-1 :
1908 : 10

Heroine :
1908 : 4

Hi-Hi :
1928 : 4

Hilarius :
1948 : 2

Hirondelle :
1952 : 2

Hojwa :
1952 : 3

Holland :
1932 : 2

Hollandia :
1928 : 6

Honduras :
1968 : 6
1976 : 3
1984 : 10
1988 : 8
1992 : 11
1996 : 7
2000 : 20
2004 : 5
2008 : 25
2012 : 25
2016 : 23

Hong Kong :
1952 : 4
1956 : 2
1960 : 4
1964 : 36
1968 : 11
1972 : 10
1976 : 25
1984 : 47
1988 : 47
1992 : 38
1996 : 23
2000 : 31
2002 : 2
2004 : 30
2006 : 1
2008 : 34
2010 : 1
2012 : 41
2014 : 1
2016 : 37

Hong Kong-1 :
1988 : 2
2004 : 4

Hong Kong-2 :
1988 : 2
2004 : 4

Hornet :

1988 : 25
1992 : 14
1994 : 10
1996 : 2
1998 : 8
2000 : 2
2002 : 8
2004 : 1
2006 : 5
2008 : 2
2010 : 4
2012 : 3
2014 : 3
2016 : 3

Life boat naval ship "Spetsai"-1 :
1906 : 7

Linglom :
1964 : 3

Lithuania :
1924 : 13
1928 : 13
1992 : 53
1994 : 6
1996 : 61
1998 : 7
2000 : 61
2002 : 8
2004 : 59
2006 : 7
2008 : 71
2010 : 6
2012 : 62
2014 : 9
2016 : 67

Liverpool Police Team-2 :
1908 : 8

Llanoria :
1948 : 5
1952 : 6

London City Police-1 :
1908 : 8

Lotta IV :
1948 : 2
1952 : 2

Lucky :
1964 : 2

Lucky Girl-1 :
1912 : 5

Lucky Star :
1948 : 2
1952 : 2
2008 : 1

Ludwigshafener Ruder Verein-1 :
1900 : 5

Ludwigshafener Ruderverein :
1912 : 5

Lully II :
1936 : 5

Luxembourg :
1900 : 1
1912 : 21
1920 : 25
1924 : 48
1928 : 53
1932 : 3
1936 : 50
1948 : 45
1952 : 44
1956 : 11
1960 : 52
1964 : 12
1968 : 5
1972 : 11
1976 : 8
1980 : 3
1984 : 5
1988 : 9
1992 : 7
1994 : 1
1996 : 6
1998 : 1
2000 : 7
2004 : 10
2006 : 1
2008 : 13
2012 : 9
2014 : 1
2016 : 10

Luxembourg-1 :
1936 : 2

Luxembourg-2 :
1936

2008 : 1

Ormsund Roklub-2 :
1912 : 5

Osborne Swimming Club, Manchester :
1900 : 7

Oxalis :
2008 : 1

Paka :
1936 : 2

Paka V :
1952 : 2

Pakaria :
1960 : 2

Pakistan :
1948 : 35
1952 : 38
1956 : 55
1960 : 44
1964 : 41
1968 : 15
1972 : 25
1976 : 24
1984 : 31
1988 : 30
1992 : 27
1996 : 24
2000 : 27
2004 : 26
2008 : 21
2010 : 1
2012 : 21
2014 : 1
2016 : 7

Palau :
2000 : 5
2004 : 4
2008 : 5
2012 : 5
2016 : 5

Palestine :
1996 : 1
2000 : 2
2004 : 3
2008 : 4
2012 : 5
2016 : 6

Pam :
1956 : 3

Pampero :
1948 : 3
1952 : 3
1956 : 3

Pan :
1928 : 5
1948 : 3
1952 : 3

Pan II :
1956 : 3

Pan-Hellenic Athletic Club :
1904 : 5

Panama :
1928 : 1
1948 : 1
1952 : 1
1960 : 6
1964 : 10
1968 : 16
1972 : 7
1976 : 8
1984 : 8
1988 : 6
1992 : 5
1996 : 7
2000 : 6
2004 : 4
2008 : 5
2012 : 8
2016 : 10

Pandora :
1964 : 2

Panhellenios Gymnastikos Syllogos-1 :
1906 : 6

Pannonia RC/National RC :
1908 : 9

Papua New Guinea :
1976 : 6
1984 : 7
1988 : 11
1992 : 13
1996 : 11
2000 : 5
2004 : 4
2008 : 7
2012 : 8
2

1984 : 3
1988 : 4
1992 : 1
1996 : 4
2000 : 2
2004 : 2
2008 : 3
2012 : 4
2016 : 3

Solos Carex :
2008 : 1

Somalia :
1972 : 3
1984 : 7
1988 : 5
1996 : 4
2000 : 2
2004 : 2
2008 : 2
2012 : 2
2016 : 2

Sonja II :
1912 : 3

Sorais-2 :
1908 : 5

Souriceau-38 :
1900 : 1

Souriceau-4 :
1900 : 1

South Africa :
1904 : 3
1908 : 14
1912 : 21
1920 : 38
1924 : 30
1928 : 24
1932 : 11
1936 : 32
1948 : 35
1952 : 61
1956 : 45
1960 : 57
1992 : 93
1994 : 2
1996 : 84
1998 : 2
2000 : 127
2002 : 1
2004 : 106
2006 : 3
2008 : 134
2010 : 2
2012 : 124
2016 : 135

South Africa-1 :
1920 : 2

South Africa-2 :
1920 : 2

South Korea :
1948 : 49
1952 : 19
1956 : 39
1960 : 42
1964 : 161
1968 : 62
1972 : 47
1976 : 53
1980 : 10
1984 : 190
1988 : 422
1992 : 239
1994 : 21
1996 : 292
1998 : 37
2000 : 273
2002 : 46
2004 : 253
2006 : 40
2008 : 255
2010 : 45
2012 : 244
2014 : 63
2016 : 199

South Korea-1 :
1988 : 2
1992 : 8
1996 : 10
2000 : 9
2004 : 9
2008 : 5
2012 : 4
2014 : 4
2016 : 4

South Korea-2 :
1988 : 2
1992 : 8
1996

1904 : 2
1924 : 4
1928 : 7
1932 : 8
1936 : 8
1948 : 8
1952 : 6
1956 : 8
1960 : 2
1964 : 10
1968 : 8
1972 : 11
1976 : 10
1980 : 12
1984 : 12
1988 : 11
1992 : 14
1994 : 10
1996 : 4
1998 : 10
2000 : 8
2002 : 12
2004 : 10
2006 : 12
2008 : 8
2010 : 12
2012 : 9
2014 : 12
2016 : 8

United States-10 :
1904 : 2

United States-11 :
1904 : 2

United States-12 :
1904 : 2

United States-13 :
1904 : 2

United States-14 :
1904 : 2

United States-2 :
1904 : 2
1924 : 4
1928 : 7
1932 : 8
1936 : 7
1948 : 8
1952 : 8
1956 : 8
1960 : 2
1964 : 10
1968 : 8
1972 : 11
1976 : 12
1980 : 10
1984 : 12
1988 : 10
1992 : 14
1994 : 8
1996 : 4
1998 : 10
2000 : 8
2002 : 13
2004 : 10
2006 : 12
2008 : 8
2010 : 12
2012 : 9
2014 : 14
2016 : 8

United States-3 :
1904 : 2
1960 : 2
1964 : 2
1968 : 2
1972 : 2
1976 : 2
1984 : 4
1988 : 2
1992 : 4
1994 : 2
1996 : 4
2006 : 2
2010 : 8
2014 : 6

United States-4 :
1904 : 2

United States-5 :
1904 : 2

United States-6 :
1904 : 2

United States-7 :
1904 : 2

United States-8 :
1904 : 2

U

To return to our athletes that share names:

In [37]:
for name, name_group in data.groupby("Name"):
    if len(name_group['ID'].unique()) > 1:
        print(name)
        print(name_group[ ['ID', 'Age', 'Team', 'Year'] ])
        print()

Abdul Aziz
         ID   Age      Team  Year
11607  6325  21.0  Pakistan  1956
11608  6326  27.0  Pakistan  1952
11609  6326  27.0  Pakistan  1952
11610  6326  27.0  Pakistan  1952
11611  6326  32.0  Pakistan  1956
11612  6326  32.0  Pakistan  1956

Abdul Hamid
          ID   Age      Team  Year
89288  45252  21.0  Pakistan  1948
89289  45252  25.0  Pakistan  1952
89290  45252  29.0  Pakistan  1956
89291  45252  33.0  Pakistan  1960
89292  45253  22.0  Pakistan  1964

Abdul Rashid
           ID   Age      Team  Year
197128  98967  29.0  Pakistan  2008
197129  98968  21.0  Pakistan  1968
197130  98968  25.0  Pakistan  1972
197131  98968  29.0  Pakistan  1976
197132  98969  24.0  Pakistan  1952
197133  98969  28.0  Pakistan  1956

Abdul Rehman
           ID   Age      Team  Year
198418  99625  23.0  Pakistan  1952
198419  99625  23.0  Pakistan  1952
198420  99626  26.0  Pakistan  1964

Abdullah Ali
        ID   Age     Team  Year
4669  2658  28.0    Egypt  1964
4670  2658  28.0    Egypt 

Balbir Singh
            ID   Age   Team  Year
220981  111012  26.0  India  1968
220982  111013  23.0  India  1968
220983  111014  23.0  India  1968

Baldev Singh
            ID   Age   Team  Year
220984  111015  24.0  India  1976
220985  111016   NaN  India  1948
220986  111016   NaN  India  1948
220987  111017  29.0  India  1980

Balzs Kiss
           ID   Age     Team  Year
120193  60834  33.0  Hungary  2016
120194  60835  24.0  Hungary  1996

Bandu Patil
           ID   Age   Team  Year
183118  92038  28.0  India  1964
183119  92039  22.0  India  1964
183120  92039  22.0  India  1964

Benjamin John "Ben" Sharpe
            ID   Age           Team  Year
217647  109323  20.0         Canada  1936
217648  109324  27.0  Great Britain  2000

Bent Larsen
           ID   Age     Team  Year
132679  66775  22.0  Denmark  1964
132680  66776  22.0  Denmark  1976

Biser Georgiev
          ID   Age      Team  Year
77965  39590  22.0  Bulgaria  1996
77966  39591  27.0  Bulgaria  1980

Bjrgvin Bjr

Daniel Radu
           ID   Age     Team  Year
195628  98213  20.0  Romania  1980
195629  98214  22.0  Romania  1980

Daniel Winkler
            ID   Age          Team  Year
261815  130995  22.0  West Germany  1984
261816  130995  22.0  West Germany  1984
261817  130995  22.0  West Germany  1984
261818  130995  22.0  West Germany  1984
261819  130995  22.0  West Germany  1984
261820  130995  22.0  West Germany  1984
261821  130995  22.0  West Germany  1984
261822  130995  22.0  West Germany  1984
261823  130995  26.0  West Germany  1988
261824  130995  26.0  West Germany  1988
261825  130995  26.0  West Germany  1988
261826  130995  26.0  West Germany  1988
261827  130995  26.0  West Germany  1988
261828  130995  26.0  West Germany  1988
261829  130995  26.0  West Germany  1988
261830  130995  26.0  West Germany  1988
261831  130996  25.0   Switzerland  1984

David "Dave" Anderson
        ID   Age           Team  Year
6784  3806  22.0         Canada  2002
6785  3807  22.0  Great Britai

Florian Vogel
            ID   Age         Team  Year
253701  127040  26.0  Switzerland  2008
253702  127040  30.0  Switzerland  2012
253703  127041  21.0      Germany  2016
253704  127041  21.0      Germany  2016

Franc Smolej
            ID   Age        Team  Year
223891  112468  23.0  Yugoslavia  1964
223892  112468  27.0  Yugoslavia  1968
223893  112469  27.0  Yugoslavia  1936
223894  112469  27.0  Yugoslavia  1936
223895  112469  27.0  Yugoslavia  1936
223896  112469  39.0  Yugoslavia  1948

Francesco Scuderi
            ID   Age   Team  Year
215246  108054  22.0  Italy  2000
215247  108054  22.0  Italy  2000
215248  108055  23.0  Italy  1972

Francisco Gonzlez Suaste
          ID   Age    Team  Year
81773  41516   NaN  Mexico  1948
81777  41518  21.0  Mexico  1968

Francisco Rodrguez
            ID   Age         Team  Year
202596  101756   NaN    Argentina  1928
202598  101758  26.0  Puerto Rico  1996

Franco Zucchi
            ID   Age        Team  Year
270681  135370  42.0  Vol

Gyula Tth
            ID   Age     Team  Year
242348  121394  29.0  Hungary  1956
242349  121394  29.0  Hungary  1956
242350  121394  33.0  Hungary  1960
242351  121395  32.0  Hungary  1968
242352  121395  36.0  Hungary  1972

Gza Tth
            ID   Age     Team  Year
242337  121391  26.0  Hungary  1988
242338  121392  21.0  Hungary  1928
242339  121392  21.0  Hungary  1928
242340  121392  21.0  Hungary  1928
242341  121392  21.0  Hungary  1928
242342  121392  21.0  Hungary  1928
242343  121392  21.0  Hungary  1928
242344  121392  21.0  Hungary  1928
242345  121393  28.0  Hungary  1960
242346  121393  32.0  Hungary  1964
242347  121393  36.0  Hungary  1968

Hans Baumann
         ID   Age            Team  Year
16302  8758  26.0         Austria  1936
16303  8758  26.0         Austria  1936
16304  8759  31.0     Switzerland  1936
16305  8760  35.0  West Germany-2  1968

Hans Huber
           ID   Age         Team  Year
100233  50710  30.0      Germany  1964
100234  50711  29.0  Switzerl

...

Peng Bo
           ID   Age   Team  Year
185038  93004  22.0  China  1984
185039  93005  23.0  China  2004
185040  93005  23.0  China  2004

Per Olsen
           ID   Age    Team  Year
176831  88905  17.0  Norway  1952
176832  88905  17.0  Norway  1952
176833  88906  23.0  Norway  1956
176834  88906  23.0  Norway  1956

Petar Angelov
        ID   Age      Team  Year
7722  4301  20.0  Bulgaria  1964
7723  4301  20.0  Bulgaria  1964
7724  4301  20.0  Bulgaria  1964
7725  4301  24.0  Bulgaria  1968
7726  4301  24.0  Bulgaria  1968
7727  4301  24.0  Bulgaria  1968
7728  4302  39.0  Bulgaria  1936
7729  4302  39.0  Bulgaria  1936

Petar Dimitrov Georgiev
          ID   Age      Team  Year
77997  39611  23.0  Bulgaria  1988
77998  39611  23.0  Bulgaria  1988
77999  39611  23.0  Bulgaria  1988
78000  39611  23.0  Bulgaria  1988
78001  39611  23.0  Bulgaria  1988
78002  39611  23.0  Bulgaria  1988
78003  39611  23.0  Bulgaria  1988
78004  39611  23.0  Bulgaria  1988
78007  39613  22.0  Bulgari

Roland Fischnaller
          ID   Age   Team  Year
69818  35555  21.0  Italy  2002
69819  35555  25.0  Italy  2006
69820  35555  29.0  Italy  2010
69821  35555  33.0  Italy  2014
69822  35555  33.0  Italy  2014
69823  35556  26.0  Italy  2002
69824  35556  26.0  Italy  2002

Ronald Gray
          ID   Age           Team  Year
83700  42544  64.0  Great Britain  1932
83701  42545  24.0      Australia  1956

Rudolf Schmid
            ID   Age           Team  Year
213193  107051  30.0  Switzerland-1  1976
213194  107052  20.0        Austria  1972
213195  107052  20.0      Austria-2  1972
213196  107052  24.0        Austria  1976
213197  107052  24.0      Austria-1  1976

Ryan Bailey
         ID   Age           Team  Year
12692  6905  23.0  United States  2012
12693  6905  23.0  United States  2012
12694  6906  25.0  United States  2000
12695  6906  28.0  United States  2004
12696  6906  32.0  United States  2008
12697  6906  36.0  United States  2012

Salim Sherwani
            ID   Age   

Tetsuo Sato
            ID   Age   Team  Year
211155  106040  19.0  Japan  1968
211156  106040  23.0  Japan  1972
211157  106040  27.0  Japan  1976
211158  106041  21.0  Japan  1960

Thomas Schneider
            ID   Age     Team  Year
213658  107294  23.0  Germany  2012
213659  107295  24.0  Germany  1956

Tian Liang
            ID   Age   Team  Year
239892  120212  16.0  China  1996
239893  120212  21.0  China  2000
239894  120212  21.0  China  2000
239895  120212  24.0  China  2004
239896  120212  24.0  China  2004
239897  120213  22.0  China  2008
239898  120213  26.0  China  2012

Tnde Szab
            ID   Age     Team  Year
233983  117354  27.0  Hungary  2016
233984  117355  18.0  Hungary  1992
233985  117355  18.0  Hungary  1992

Todor Todorov
            ID   Age        Team  Year
240918  120686  26.0  Bulgaria-2  1988
240919  120687  28.0    Bulgaria  1976

Tom Kuera
           ID   Age            Team  Year
128227  64570  23.0  Czech Republic  2000
128228  64571  19.0  Czech

Werner Fehr
          ID  Age         Team  Year
67402  34453  NaN  Switzerland  1924
67403  34453  NaN  Switzerland  1924
67404  34454  NaN  Switzerland  1928

Werner Schmid
            ID   Age         Team  Year
213209  107060  27.0  Switzerland  1960
213210  107061  28.0  Switzerland  1948
213211  107061  32.0  Switzerland  1952
213212  107061  32.0  Switzerland  1952

Wilhelm Kaufmann
           ID   Age     Team  Year
114690  58076  36.0  Germany  1908
114691  58077  53.0  Austria  1948

William "Bill" Robinson
            ID   Age           Team  Year
201935  101416  27.0         Canada  1976
201936  101417  28.0  Great Britain  1964

William Hill
          ID   Age         Team  Year
96000  48602  19.0    Hong Kong  1964
96001  48602  19.0    Hong Kong  1964
96002  48603   NaN  Australasia  1908
96003  48603   NaN  Australasia  1908
96004  48603   NaN  Australasia  1908

William Wilson
            ID   Age           Team  Year
261620  130906  43.0  Great Britain  1948
261622  1

Zhao Wei
            ID   Age   Team  Year
269123  134668  21.0  China  2000
269124  134669  25.0  China  1988

Zhou Peng
            ID   Age   Team  Year
269399  134805  24.0  China  2008
269400  134805  28.0  China  2012
269401  134806  22.0  China  2012
269402  134806  26.0  China  2016

Zhou Yan
            ID   Age   Team  Year
269443  134821  27.0  China  2010
269444  134821  31.0  China  2014
269445  134822  21.0  China  2000

Zhou Yang
            ID   Age   Team  Year
269446  134823  20.0  China  2008
269447  134824  18.0  China  2010
269448  134824  18.0  China  2010
269449  134824  18.0  China  2010
269450  134824  18.0  China  2010
269451  134824  22.0  China  2014
269452  134824  22.0  China  2014

Zhu Ting
            ID   Age   Team  Year
269511  134857  21.0  China  2016
269512  134858  23.0  China  2008

Zoltn Horvth
          ID   Age     Team  Year
98983  50103  23.0  Hungary  1960
98984  50103  23.0  Hungary  1960
98985  50103  27.0  Hungary  1964
98986  50104  25.

## Aggregating and Plotting

In [40]:
data['Year'].max()

2016